In [ ]:
!pip install awswrangler

In [1]:
import boto3
import awswrangler as wr
import json

In [2]:
rdshft = boto3.client('redshift')
scm = boto3.client('secretsmanager')

In [3]:
# configuration
rdshft_cluster='redshift-beyoung-20200922-10-07'
role_arn=rdshft.describe_clusters(ClusterIdentifier=rdshft_cluster)['Clusters'][0]['IamRoles'][0]['IamRoleArn'];

In [4]:
response = scm.get_secret_value(SecretId=rdshft_cluster)
dbInfo = json.loads(response['SecretString']);

In [18]:
eng_redshift = wr.db.get_engine(db_type='redshift',host=dbInfo['host'],port=dbInfo['port'],
                          database=dbInfo['dbname'],user=dbInfo['username'],password=dbInfo['password'])
conn = eng_redshift.connect()

# create tables and insert into data

In [7]:
sql_acc='''
DROP TABLE IF EXISTS public.accommodations;
CREATE TABLE public.accommodations (
  id INTEGER PRIMARY KEY,
  shape GEOMETRY,
  name VARCHAR(100),
  host_name VARCHAR(100),
  neighbourhood_group VARCHAR(100),
  neighbourhood VARCHAR(100),
  room_type VARCHAR(100),
  price SMALLINT,
  minimum_nights SMALLINT,
  number_of_reviews SMALLINT,
  last_review DATE,
  reviews_per_month NUMERIC(8,2),
  calculated_host_listings_count SMALLINT, 
  availability_365 SMALLINT
);
'''

In [8]:
conn.execute(sql_acc);

In [7]:
sql_acc_data=f'''
COPY public.accommodations
FROM 's3://redshift-downloads/spatial-data/accommodations.csv'
DELIMITER ';'
IGNOREHEADER 1 REGION 'us-east-1'
CREDENTIALS 'aws_iam_role={role_arn}';
'''

In [8]:
conn.execute(sql_acc_data);

In [9]:
sql_zip='''
DROP TABLE IF EXISTS public.zipcode;
CREATE TABLE public.zipcode (
  ogc_field INTEGER PRIMARY KEY NOT NULL,
  wkb_geometry GEOMETRY,
  gml_id VARCHAR,
  spatial_name VARCHAR,
  spatial_alias VARCHAR,
  spatial_type VARCHAR
 );
'''

In [10]:
conn.execute(sql_zip);

In [16]:
sql_zip_data=f'''
COPY public.zipcode
FROM 's3://redshift-downloads/spatial-data/zipcode.csv'
DELIMITER ';'
IGNOREHEADER 1 REGION 'us-east-1'
CREDENTIALS 'aws_iam_role={role_arn}';
'''

In [19]:
conn.execute(sql_zip_data);

# Querying spatial data

In [20]:
sql='SELECT count(*) FROM public.accommodations WHERE ST_SRID(shape) = 4326;'
wr.db.read_sql_query(sql, con=eng_redshift)

,count
0,22248


In [ ]:
# [{column: value for column, value in rowproxy.items()} for rowproxy in resultproxy]

In [21]:
sql='''
SELECT 
  ogc_field, spatial_name, spatial_type, 
  ST_SRID(wkb_geometry), ST_AsText(wkb_geometry) 
FROM public.zipcode 
ORDER BY spatial_name;
'''

In [ ]:
wr.db.read_sql_query(sql, con=eng_redshift)

In [ ]:
SELECT 
  ogc_field, spatial_name, 
  ST_AsGeoJSON(wkb_geometry), ST_Dimension(wkb_geometry), ST_NPoints(wkb_geometry)
FROM public.zipcode 
WHERE spatial_name='10117';

In [ ]:
SELECT count(*) 
FROM public.accommodations 
WHERE ST_DWithin(shape, ST_GeomFromText('POINT(13.377704 52.516431)', 4326), 0.01);

In [ ]:
WITH poi(loc) as (
  SELECT st_astext(shape) FROM accommodations WHERE name LIKE '%brandenburg gate%'
)
SELECT count(*) 
FROM accommodations a, poi p
WHERE st_dwithin(a.shape, ST_GeomFromText(p.loc, 4326), 0.01);

In [ ]:
SELECT name, price, ST_AsText(shape) 
FROM public.accommodations
WHERE ST_DWithin(shape, ST_GeomFromText('POINT(13.377704 52.516431)', 4326), 0.01)
ORDER BY price DESC;

In [ ]:
SELECT 
  a.price, a.name, ST_AsText(a.shape), 
  z.spatial_name, ST_AsText(z.wkb_geometry) 
FROM accommodations a, zipcode z 
WHERE 
  ST_Within(a.shape, z.wkb_geometry) AND 
  price = (SELECT avg(price) FROM accommodations);

In [ ]:
SELECT z.spatial_name as zip, count(*) as numAccommodations 
FROM public.accommodations a, public.zipcode z
WHERE ST_Within(a.shape, z.wkb_geometry)
GROUP BY zip 
ORDER BY numAccommodations DESC;

# Running spatial queries on data stored in Amazon Spectrum

In [ ]:
create external schema spectrumdb from data catalog 
database 'spectrumdb' 
iam_role 'arn:aws:iam::123456789012:role/mySpectrumRole' 
create external database 
if not exists;

In [ ]:
create external table spectrumdb.geoname (
    geonameid   INT,
    name VARCHAR(200),
    asciiname VARCHAR(200),
    alternatenames TEXT,
    latitude FLOAT,
    longitude FLOAT,
    fclass CHAR(1),
    fcode VARCHAR(10),
    country VARCHAR(2),
    cc2 VARCHAR(60),
    admin1 VARCHAR(20),
    admin2 VARCHAR(80),
    admin3 VARCHAR(20),
    admin4 VARCHAR(20),
    population BIGINT,
    elevation INT,
    gtopo30 INT,
    timezone VARCHAR(40),
    moddate DATE
 )
 row format delimited fields terminated by '\t' 
 stored as textfile location 's3://yourSpectrumData/';

In [ ]:
SELECT ST_GeomFromText('POINT('||longitude||' '||latitude||')', 4326) 
FROM spectrumdb.geoname; 

In [ ]:
SELECT 
  *, 
  ST_GeomFromText('POINT('||longitude||' '||latitude||')', 4326) 
FROM 
  spectrumdb.geoname
WHERE 
  ST_DWithin(
    ST_GeomFromText('POINT('||longitude||' '||latitude||')', 4326), 
    ST_GeomFromText('POINT(13.377704 52.516431)', 4326), 
    0.01);

In [ ]:
COPY public.geoname 
FROM 's3://hinnerk/spatial/DE.csv' 
DELIMITER '\t' 
MAXERROR 1000
CREDENTIALS 'aws_iam_role=arn:aws:iam::279345015071:role/SpatialDemoRole';